# LORIS API Tour 2/2

This tutorial contains basic examples in Python to demonstrate how to interact with the API.
To run this tutorial, click on `Runtime -> Run all`This tutorial is also available as a Google colab notebook so you can run it directly from your browser. To access it, click on the button below: <a href="https://colab.research.google.com/github/aces/Loris/blob/main/docs/notebooks/LORIS_API_Part2_Python-script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
import getpass   # For input prompt hide what is entered
import json      # Provides convenient functions to handle json objects 
import re        # For regular expression
import requests  # To handle http requests
import warnings  # To ignore warnings

# Because the ssl certificates are unverified, warnings are thrown at every 
# HTTPS request. The following command will prevent warning messages to be
# printed at every HTTPS request.
warnings.simplefilter('ignore')

baseurl = 'https://demo.loris.ca/api/v0.0.3'

def prettyPrint(string):
    print(json.dumps(string, indent=2, sort_keys=True))

## About the data in demo.loris.ca
The demo instance of LORIS contains the raisinbread dataset, which is used 
by LORIS developers. For more informations about raisinbread, click [here](https://github.com/aces/Loris/tree/main/raisinbread)

## Exercise 1. Login

This is a POST request to the `/login` endpoint that requires 2 parameters: `username` and `password`  
The expected response is a json string that contains a token property.  

https://github.com/aces/Loris/blob/main/modules/api/docs/LorisRESTAPI.md

In [ ]:
payload = {
    'username': input('username: '), 
    'password': getpass.getpass('password: ')
}

response = requests.post(
    url = '/'.join([baseurl, 'login']),
    json = payload,
    verify = False
)

text = response.content.decode('ascii')

data = json.loads(text)

prettyPrint(data)

*Store the token in a variable for later*

In [ ]:
token = data['token']

The requests thoughout this tutorial are very similar, only the URL changes. We will define 5 functions that will make the tutorial easier to read.

For more information on HTTP requests and their differences, you can refer to https://github.com/aces/Loris/blob/main/modules/api/docs/LorisRESTAPI.md#10-overview

In [ ]:
def GET(url):
    """
    Function to send an HTTP GET request

    Parameters: 
        url :   url where the GET request is sent

    Returns: projects (Information on all projects)
    """
    response = json.loads(requests.get(
        url = url,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    ).content.decode('ascii'))
    return response

def GETFile(url):
    """
    Function to send an HTTP GET request to download files

    Parameters: 
        url :   url where the GET request is sent

    Returns: projects (Information on all projects)
    """
    filename = file['Filename']
    response = requests.get(
        url = url,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    )
    f = open(filename, "w+b")
    f.write(bytes(response.content))
    return response

def PUT(url, json_input):
    """
    Function to send an HTTP PUT request

    Parameters: 
        url :           url where the GET request is sent
        json_input :    json object to query the information for the PUT request

    """
    response = requests.put(
        url = url,
        json = json_input,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    )
    return response

def PATCH(url, json_input):
    """
    Function to send an HTTP PATCH request

    Parameters: 
        url :           url where the GET request is sent
        json_input :    json object to query the information for the PATCH request

    """
    response = requests.patch(
        url = url,
        json = json_input,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    )
    return response

def POST(url, json_input):
    """
    Function to send an HTTP POST request

    Parameters: 
        url :           url where the GET request is sent
        json_input :    json object to query the information for the PATCH request

    """
    response = requests.post(
        url = url,
        json = json_input,
        verify = False,
        headers = {'Authorization': 'Bearer %s' % token}
    )
    return response



# Exercice 2 - Projects

The endpoints in this section are used to get information on all candidates, to get information on a specific candidate or to add a new candidate.

https://github.com/aces/Loris/blob/master/modules/api/docs/LorisRESTAPI.md#20-project-api

## Exercise 2.1 Get all projects

The endpoint `/projects` can be used to obtain information on all the projects contained in the database.


In [ ]:
# Save the project names to test other endpoints later
url = '/'.join([baseurl, 'projects'])
projects = GET(url)
projectNames = list(projects['Projects'].keys())

prettyPrint(projects)

## Exercise 2.2 Get a specific project

The endpoint `/projects/{project}` can be used to obtain information on a specific project contained in the database.


In [ ]:
url = '/'.join([baseurl, 'projects', projectNames[0]])
project = GET(url)
prettyPrint(project)

## Exercise 2.3 Get all the candidates in a specific project

The endpoint `/projects/{project}/candidates` can be used to obtain information on all the candidates for a specific project.


In [ ]:
url = '/'.join([baseurl, 'projects', projectNames[0], 'candidates'])
projectCandidates = GET(url)
prettyPrint(projectCandidates)

## Exercise 2.4 Get all the images in a specific project

The endpoint `/projects/{project}/images` can be used to obtain information on all the images that are used in a project.


In [ ]:
url = '/'.join([baseurl, 'projects', projectNames[0], 'images'])
projectImages = GET(url)
prettyPrint(projectImages)

### Find recent images

In [ ]:
url = '/'.join([baseurl, 'projects', projectNames[0],
                'images?since=2018-12-13T10:20:18-05:00'])
projectRecentImages = GET(url)
prettyPrint(projectRecentImages)

## Exercise 2.5 Get all the instruments in a specific project

The endpoint `/projects/{project}/instruments` can be used to obtain information on all the instruments that are used in a project.


In [ ]:
url = '/'.join([baseurl, 'projects', projectNames[0], 'instruments'])
projectInstruments = GET(url)
prettyPrint(projectInstruments)

## Exercise 2.6 Get a specific instruments in a specific project

The endpoint `/projects/{project}/instruments/{instrument}` can be used to obtain information on a specific instrument that is used in a specific project.


In [ ]:
singleInstrument = list(projectInstruments['Instruments'].keys())[1]
url = '/'.join([baseurl, 'projects', projectNames[0],
                'instruments', singleInstrument])
projectSingleInstrument = GET(url)
prettyPrint(projectSingleInstrument)

## Exercise 2.7 Get all the electrophysiology recordings in a specific project

The endpoint `/projects/{project}/images` can be used to obtain information on all the images that are used in a project.


In [ ]:
url = '/'.join([baseurl, 'projects', projectNames[0], 'recordings'])
projectRecordings = GET(url)
prettyPrint(projectRecordings)

# Exercice 3 - Candidates

The endpoints in this section are used to get information on all candidates, to get information on a specific candidate or to add a new candidate.

https://github.com/aces/Loris/blob/master/modules/api/docs/LorisRESTAPI.md#30-candidate-api

## Exercise 3.1 Get all candid

The endpoint `/candidates` can be used to obtain information on all the candidates contained in the database.


In [ ]:
url = '/'.join([baseurl, 'candidates'])
allCandidates = GET(url)
prettyPrint(allCandidates)

## Exercise 3.2 Create a candidate
 - Send a **POST** request to /candidates with a payload containing an object with a candidate property
 ```json
    "Candidate" : {
        "Project" : ProjectName,
        "PSCID"   : PSCID, # only if config is set to prompt 
        "EDC"     : "YYYY-MM-DD", # optional
        "DoB"     : "YYYY-MM-DD",
        "Sex"     : "Male|Female",
        "Site"    : SiteName,
    }
 ```
 

In [ ]:
# Keep some variables for the next examples
projectname = list(projects['Projects'].keys())[0]
sitename = allCandidates['Candidates'][0]['Site']

In [ ]:
json_data = {
    'Candidate' : {
        'Project' : projectname,
        'DoB'     : "2015-09-10",
        'EDC'     : "2015-09-10", #Optional
        'Sex'     : "Female",
        'Site'    : sitename,
    }
}
url = '/'.join([baseurl, 'candidates'])
response = POST(url, json_data)
print('POST response status code:', response.status_code)


GET request to verify the new candidate is actually created

In [ ]:
newCandidate = GET('/'.join([baseurl, 'candidates', json.loads(response.content)['CandID']]))
prettyPrint(newCandidate)

## Exercise 3.3 Get a specific candidate

This is a GET request to /candidates/{candid}


In [ ]:
candid = allCandidates['Candidates'][0]['CandID']
singleCandidate = GET('/'.join([baseurl, 'candidates', candid]))
prettyPrint(singleCandidate)
# keep a visit for the next examples
candidateVisit = singleCandidate['Visits'][0]

# Exercice 4 - Visits

The endpoints in this section are used to get information on all candidates, to get information on a specific candidate or to add a new candidate.

https://github.com/aces/Loris/blob/master/modules/api/docs/LorisRESTAPI.md#40-imaging-data

## Exercise 4.1 Get all visits for a specific candidate

This is a GET request to /candidates/{candid}/{visit}


In [ ]:
url = '/'.join([baseurl, 'candidates', candid, candidateVisit])
candidateVisits = GET(url)
prettyPrint(candidateVisits)

## Exercise 4.2 Add a timepoint for a given candidate
 - **PUT** request to /candidates/candid/{visit}
```json
{
    "Meta" : {
        "Battery" : "Fresh",
        "CandID" : candid,
        "Project" : "Pumpernickel",
        "Visit"  : visit,
        "Battery": "NameOfSubproject"
    "Stages": {
        "Visit": {
            "Date": "2017-03-26",
        }
    }
}
```

 - All VisitLabels for a given candidates can be found using /candidates/\$candid/
 
 
 - Every possible visit_labels for a project can be found using /projects/\$projectname/visits. If a {visit} is in /projects/{project}/visits but not in /candidates/{candid}/ , it can be added with a **PUT** request to /candidates/\$candid/\$visit_label


 - Battery (NameOfSubproject) must be guessed... 
 
 

In [ ]:
json_data = {
        "Battery" : 'Stale',
        "Site" : 'Data Coordinating Center',
        "CandID" : '400266',
        "Visit"  : 'V3',
        "Project": 'Pumpernickel'
}
url = '/'.join([baseurl, '/candidates/400266/V3'])
response = PUT(url, json_data)
print('PUT response status code:', response.status_code)

In [ ]:
editedVisit = GET(url)
prettyPrint(editedVisit)

## Exercise 4.3 Get QC/imaging of a candidate visit

This is a GET request to /candidates/{candid}/{visit}


In [ ]:
url = '/'.join([baseurl, 'candidates', candid, candidateVisit, 'qc', 'imaging'])
singleCandidateQcImaging = GET(url)
prettyPrint(singleCandidateQcImaging)

## Exercise 4.4 Change QC/imaging of a candidate visit
 - **PUT** request to /candidates/{candid}/{visit}/ qc/imaging
```json
{
    "Meta" : {
        "CandID" : candid,
        "Visit"  : visit,
        "Battery": "NameOfSubproject"
}
```

 - All VisitLabels for a given candidates can be found using /candidates/{candid}/
 
 
 - Every possible {visit} labels for a project can be found using /projects/{project}/visits. If a {visit} label is in /projects/{project}/visits but not in /candidates/{candid}/ , it can be added using a **PUT** request to /candidates/{candid}/{visit}


 - Battery (NameOfSubproject) must be guessed... 

In [ ]:
# In the previous example, "SessionQC" was set to true. 
# We'll change it with a PUT request

json_data = {
    'Meta' : {
        'CandID' : candid,
        'Visit'  : candidateVisit,
    },
    'SessionQC': "",
    'Pending': False
}
url = '/'.join([baseurl, 'candidates', candid, candidateVisit, 'qc', 'imaging'])
response = PUT(url, json_data)

print('PUT response status code:', response.status_code)


In [ ]:
# Pending should now be false
visitqcImaging = GET(url)
prettyPrint(visitqcImaging)

# Exercice 5: Instrument data for a candidate


## Exercise 5.1. Find all candidates and session with a given instruments

This is a series of GET request  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#31-specific-candidate  
https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-candidate-instruments   
https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

In [ ]:
request_count = 0

for candidate in allCandidates['Candidates'][:4]:
    candid = candidate['CandID']
    request_count += 1
    visit_labels = GET('/'.join([baseurl, 'candidates', candid]))['Visits']
    for visit_label in visit_labels:
        request_count += 1
        instruments = GET('/'.join([baseurl,
                                    'candidates',
                                    candid, 
                                    visit_label, 
                                    'instruments']))['Instruments']
        for instrument in instruments:
            request_count += 1
            instr = GET('/'.join([baseurl,
                                  'candidates',
                                  candid,
                                  visit_label,
                                  'instruments',
                                  instrument]))
            print(json.dumps(instr, indent=2, sort_keys=True))

print(request_count)



In [ ]:
# Get an example of candid and visit_label from the last 
# sample visited in the loop 
candid_instruments = instr['Meta']['Candidate']
visit_label_instruments = instr['Meta']['Visit']
candid_instrument = instr['Meta']['Instrument']

## Exercise 5.2 GET all instruments for a candidate


In [ ]:
url = '/'.join([baseurl, 'candidates', candid_instruments, 
                visit_label_instruments, 'instruments'])
candidateinstruments = GET(url)
prettyPrint(candidateinstruments)

## Exercise 5.3 Input instrument data for a candidate
  
GET, PUT or PATCH request to /candidates/{candid}/{visit}/instruments/{instrument}  

https://github.com/aces/Loris/blob/main/modules/api/docs/LorisRESTAPI.md#33-the-candidate-instrument-data

data format:
```json
{
  "Meta": {
    "Candidate": string,
    "Visit": string
    "DDE": true|false,
    "Instrument": string,
  },
  <instrument_name>: {
    <field1_name>: <value1>,
    <field2_name>: <value2>,
    ...
  }
}
```


### 5.3.1 GET request containing all the fields

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_instruments, 
                visit_label_instruments, 'instruments', candid_instrument])
candidateSelectedInstrument = GET(url)
prettyPrint(candidateSelectedInstrument)

### 5.3.2 PATCH request

In [ ]:
# Get all the fields an meta data
url = '/'.join([baseurl, 'candidates', candid_instruments, 
                visit_label_instruments, 'instruments', candid_instrument])
json_input = GET(url)

# Update one field
json_input[candid_instrument]['Candidate_Age'] = 3
json_input[candid_instrument]['UserID'] = 'something'

response = PATCH(url, json_input)
print('PATCH response status code:', response.status_code)

In [ ]:
# GET request to verify the Candidate_Age and UserID fields have been updated
candidateSelectedInstrument = GET(url)
prettyPrint(candidateSelectedInstrument)

In [ ]:
# Update one field
json_input[candid_instrument]['Candidate_Age'] = 42

response = PATCH(url, json_input)
print('PATCH response status code:', response.status_code)

In [ ]:
"""
GET request to verify the Candidate_Age is changed to 42 and UserID is still 
"something". PATCH requests only change the field requested should, so UserID
still be "something"
"""
candidateSelectedInstrument = GET(url)
prettyPrint(candidateSelectedInstrument)

### 5.3.3 PUT request

In [ ]:
json_input = {
    'Meta': {
        'Candidate': candid_instruments,
        'DDE': False,
        'Instrument': 'mri_parameter_form',
        'Visit': visit_label_instruments},
        candid_instrument: 
            {
                'UserID': 'something_else'
            }
}

url = '/'.join([baseurl, 'candidates', candid_instruments, 
                visit_label_instruments, 'instruments', candid_instrument])

response = PUT(url, json_input)
print('PUT response status code:', response.status_code)


In [ ]:
"""
UserID should now be something_else. Because it is a PUT request, Candidate_Age 
should be changed back to null. This is a good example to display the main 
difference between PUT and PATCH: PUT restores every value to their default 
values, except for the values explicitely modified.
"""
candidateSelectedInstrument = GET(url)
prettyPrint(candidateSelectedInstrument)

## Exercise 5.4 Input instrument flags for a candidate
  
PUT or PATCH request to /candidates/{candid}/{visit}/instruments/{instrument}  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

data format:
```json
{
  "Meta": {
    "Candidate": string,
    "Visit": string
    "DDE": false,
    "Instrument": string,
  },
  "Flags": {
    "Data_entry": string,
    "Administration": string
    "Validity": true|false
  }
}
```


### 5.4.1 GET request containing all the fields for an instrument flags

In [ ]:

url = '/'.join([baseurl, 'candidates', candid_instruments, 
                visit_label_instruments, 'instruments', candid_instrument, 'flags'])

candidateSelectedInstrumentFlags = GET(url)
prettyPrint(candidateSelectedInstrumentFlags)

#### 5.4.2 PUT request containing all the fields

In [ ]:
# Update all fields
json_input = {
        'Flags': {
            "Data_entry": "In Progress",
            "Administration": "All",
            "Validity": "Valid"
        }
}

response = PUT(url, json_input)
print('PUT response status code:', response.status_code)

In [ ]:
url

In [ ]:
# Administration should now be changed to 'None'
candidateSelectedInstrument = GET(url)
prettyPrint(candidateSelectedInstrument)

#### 5.4.3 PATCH request containing some of the fields

In [ ]:
# Update one field
json_input = {
        'Flags': {
            "Administration": "None",
        }
}

response = PATCH(url, json_input)
print('PATCH response status code:', response.status_code)

GET request to verify the Administration and Data_entry fields have been updated successfully

In [ ]:
# Administration should be 'Partial' and Data_entry should be 'Valid'
candidateSelectedInstrument = GET(url)
prettyPrint(candidateSelectedInstrument)

## Exercise 5.5 Input instrument DDE for a candidate
  
GET, PUT or PATCH request to /candidates/{candid}/{visit}/instruments/{instrument}  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

data format:
```json
{
  "Meta": {
    "Candidate": string,
    "DDE": true,
    "Instrument": string,
    "Visit": string
  },
  <instrument_name>: {
    <field1_name>: <value1>,
    <field2_name>: <value2>,
    ...
  }
}
```


#### 5.5.1 GET request containing all the fields

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_instruments, visit_label_instruments,
                'instruments', candid_instrument, 'dde'])

candidateSelectedInstrumentDde = GET(url)

prettyPrint(candidateSelectedInstrumentDde)

#### 5.5.2 PATCH request containing all the fields

In [ ]:
# Get all the fields and meta data
json_input = GET(url)
# Update one field
json_input[candid_instrument]['Candidate_Age'] = 4
json_input[candid_instrument]['UserID'] = '29'

response = PATCH(url, json_input)

print('PATCH response status code:', response.status_code)

In [ ]:
# Candidate_Age should be 4 and UserID should be 29
candidateSelectedInstrumentDde = GET(url)
prettyPrint(candidateSelectedInstrumentDde)

#### 5.5.3 PUT request containing a single field to modify

In [ ]:
## Update one field
json_input = {
    'Meta': {
        'Candidate': candid_instruments,
        'DDE': True,
        'Instrument': candid_instrument,
        'Visit': candidateVisit},
        'mri_parameter_form': {'UserID': 1}}

response = PUT(url, json_input)

print('PUT response status code:', response.status_code)

In [ ]:
# UserID should be changed to "1". 
# Because it is a PUT request, Candidate_Age should be back to the default: null
candidateSelectedInstrumentDde = GET(url)
prettyPrint(candidateSelectedInstrumentDde)

## Exercise 5.6 Input instrument DDE flags for a candidate
  
PUT or PATCH request to /candidates/{candid}/{visit}/instruments/{instrument}  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#33-the-candidate-instrument-data

data format:
```json
{
  "Meta": {
    "Candidate": string,
    "Visit": string
    "DDE": true,
    "Instrument": string,
  },
  "Flags": {
    "Data_entry": string,
    "Administration": string
    "Validity": true|false
  }
}
```


### 5.6.1 GET request to query all the fields

In [ ]:
url = '/'.join([baseurl, 'candidates', candid, visit_label_instruments,
                'instruments', instrument, 'dde', 'flags'])
candidateSelectedInstrumentDdeFlags = GET(url)

prettyPrint(candidateSelectedInstrumentDdeFlags)

### 5.6.2 PATCH request

In [ ]:
json_input = {
        'Flags': {
            "Administration": "Partial",
        }
}

response = PATCH(url, json_input)

print('PATCH response status code:', response.status_code)

In [ ]:
# Data_entry should be 'Complete'
candidateSelectedInstrumentDdeFlags = GET(url)
prettyPrint(candidateSelectedInstrumentDdeFlags)

### 5.6.3 PUT request

In [ ]:
json_input = {
        'Flags': {
            "Data_entry": "In Progress",
            "Administration": "Partial",
            "Validity": "Valid"
        }
}

response = PUT(url, json_input)

print('PUT response status code:', response.status_code)

In [ ]:
# Administration shoud be 'All'. 
candidateSelectedInstrumentDdeFlags = GET(url)
prettyPrint(candidateSelectedInstrumentDdeFlags)

# Exercice 6: Images data for a candidate

In [ ]:
candid_img = projectImages['Images'][0]['Candidate']
visit_img = projectImages['Images'][0]['Visit']
imagename = projectImages['Images'][0]['Link'].split('/')[-1]

## Exercise 6.1 Find all images of a candidate for a visit

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_img, visit_img, 'images'])
candidateImages = GET(url)
prettyPrint(candidateImages)

## Exercise 6.2 Download all minc files of a candidate

In [ ]:
for file in candidateImages['Files']:
    filename = file['Filename']
    url = '/'.join([baseurl, 'candidates', candid_img, 
                    visit_img, 'images', filename])
    response = GETFile(url)
    print('GET response status code for ' + filename + ':', response.status_code)


## Exercise 6.3 GET QC data for a MINC image file

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_img, visit_img, 'images',
                imagename, 'qc'])
candidateImagesQc = GET(url)
prettyPrint(candidateImagesQc)

## Exercise 6.4 PUT QC data for a MINC image file

PUT request to /candidates/$CandID/$VisitLabel/images/$imagename/qc  

https://github.com/aces/Loris/blob/minor/docs/API/LorisRESTAPI.md#

data format:
```json
{
            'Meta'     : {
                'CandID' : string,
                'Visit'  : string,
                'File'   : string
            },
            "QC"       : string|null,
            "Selected" : boolean,
            'Caveats'  : {
                '0' : {
                    'Severity'   : string|null,
                    'Header'     : string|null,
                    'Value'      : string|null,
                    'ValidRange' : string|null,
                    'ValidRegex' : string|null
                    }
                }
}
```


In [ ]:
json_input = GET(url)
json_input['Selected'] = False

response = PUT(url, json_input)

print('PUT response status code:', response.status_code)

In [ ]:
candidateSelectedInstrumentDdeFlags = GET(url)
prettyPrint(candidateSelectedInstrumentDdeFlags)

## Exercise 6.5 GET Image formats

### Exercise 6.5.1 GET Image Brainbrowser format

In [ ]:
candidateImageFormatBrainbrowser = GET('/'.join([baseurl, 'candidates',
                                                 candid_img, visit_img, 
                                                 'images', imagename, 
                                                 'format', 'brainbrowser'])) 
prettyPrint(candidateImageFormatBrainbrowser)

### Exercise 6.5.2 GET Image Raw format

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_img, visit_img, 
                'images', imagename, 'format', 'raw'])

response = GETFile(url)

print('GET response status code', response.status_code)


### Exercise 6.5.3 GET Image Thumbnail format

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_img, visit_img, 
                        'images', imagename, 'format', 'thumbnail'])

response = GETFile(url)

print('GET response status code', response.status_code)


## Exercise 6.6 GET Image headers

### Exercise 6.6.1 GET Image headers

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_img,
                visit_img, 'images', imagename, 'headers'])
candidateImageHeaders = GET(url)
prettyPrint(candidateImageHeaders)

### Exercise 6.6.2 GET Image headers full

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_img,
                visit_img, 'images', imagename, 'headers', 'full'])
candidateImageHeadersFull = GET(url)
prettyPrint(candidateImageHeadersFull)

### Exercise 6.6.3 GET Image headers headername

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_img, visit_img, 
                'images', imagename, 'headers', 'specific'])
candidateImageHeadersHeadername = GET(url)
prettyPrint(candidateImageHeadersHeadername)

# Exercice 7: Electrophysiology recordings data for a candidate

In [ ]:
# First, find a candidate visit with an electrophysiology recording
candid_recs = projectRecordings['Recordings'][0]['Candidate']
visit_recs = projectRecordings['Recordings'][0]['Visit']
recording_filename = projectRecordings['Recordings'][0]['Link'].split('/')[-1]

## Exercise 7.1 Find all recordings of a candidate for a visit

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 'recordings'])
candidateRecordings = GET(url)
prettyPrint(candidateRecordings)

## Exercise 7.2 Download electrophysiology recording (edf) files

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 
                'recordings', recording_filename])

response = GETFile(url)

print('GET response status code', response.status_code)

## Exercise 7.3 Get electrophysiology recording channels data

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 
                'recordings', recording_filename, 'channels'])
candidateRecordingsChannels = GET(url)
prettyPrint(candidateRecordingsChannels)

## Exercise 7.4 Get electrophysiology recording channels meta data

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 'recordings', 
                recording_filename, 'channels', 'meta'])
candidateRecordingsChannelsMeta = GET(url)
prettyPrint(candidateRecordingsChannels)

## Exercise 7.5 Get electrophysiology recording electrodes data

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 'recordings', 
                recording_filename, 'electrodes'])
candidateRecordingsElectrodes = GET(url)
prettyPrint(candidateRecordingsChannels)

## Exercise 7.6 Get electrophysiology recording electrodes meta data

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 'recordings', 
                recording_filename, 'electrodes', 'meta'])
candidateRecordingsElectrodesMeta = GET(url)
prettyPrint(candidateRecordingsChannels)

## Exercise 7.7 Get electrophysiology recording events data

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 
                'recordings', recording_filename, 'events'])
candidateRecordingsEvents = GET(url)
prettyPrint(candidateRecordingsEvents)

## Exercise 7.8 Get electrophysiology recording events meta data

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_recs, visit_recs, 'recordings', 
                recording_filename, 'events', 'meta'])
candidateRecordingsEventsMeta = GET(url)
prettyPrint(candidateRecordingsEventsMeta)

# Exercise 8 - Dicoms

In [ ]:
# The flag is used to break the loops once an example that contains a 
# Dicom file is found
flag = 0

for candidate in allCandidates['Candidates']:
    candid = candidate['CandID']
    visit_labels = GET('/'.join([baseurl, 'candidates', candid]))['Visits']
    if flag == 1:
        break
    candid = candidate['CandID']
    visit_labels = GET('/'.join([baseurl, 'candidates', candid]))['Visits']
    for visit_label in visit_labels:
        dicoms = GET('/'.join([baseurl, 'candidates', candid, 
                               visit_label, 'dicoms']))
        # We only want to get a single valid example of a candidate and visit that
        # has Dicom files that can be used to test the Dicoms endpoints 
        if len(dicoms['DicomTars']) > 0:
            flag = 1
            break


## Exercise 8.1 GET a list of Dicom files for a candidate

In [ ]:
# Get an example of candid, visit_label and dicom tarname from the last 
# sample visited in the loop 
candid_dicom = dicoms['Meta']['CandID']
dicom_visit = dicoms['Meta']['Visit']
dicom_name = dicoms['DicomTars'][0]['Tarname']

In [ ]:
url = '/'.join([baseurl, 'candidates', candid_dicom, dicom_visit, 'dicoms'])
candidateDicoms = GET(url)
prettyPrint(candidateRecordingsEventsMeta)

## Exercise 8.2 Download a Dicom tar file

In [ ]:
print('Downloading file', dicom_name)
response = GETFile('/'.join([baseurl, 'candidates', candid_dicom, dicom_visit, 'dicoms', dicom_name]))
print('GET response status code:', response.status_code)